In [1]:
import casadi
import numpy as np

import ncsudyn

In [2]:
class QuadrupedDynamics(ncsudyn.dynamics.Dynamics):
    def __init__(self, m, I, L, leg_length):
        super().__init__()
        self.nq = 3 + 4
        self.nv = 3
        self.nu = 2 + 2 + 4
        self.g = 9.81
        self.m = m
        self.I = I
        self.L = L
        self.leg_length = leg_length

        cq = casadi.SX.sym("q", self.nq)  # generalized position
        cv = casadi.SX.sym("v", self.nv)  # generalized velocity
        cu = casadi.SX.sym("u", self.nu)  # control input

        F1_x, F1_y, F2_x, F2_y = cu[0], cu[1], cu[2], cu[3]
        ax = (F1_x + F2_x) / m
        ay = (F1_y + F2_y) / m - self.g  # gravity

        com_x, com_y = self.com_position(cq)
        rfront_x, rfront_y = self.front_leg_position(cq)
        rrear_x, rrear_y = self.rear_leg_position(cq)

        r_front_diff = np.array([rfront_x - com_x, rfront_y - com_y])
        r_rear_diff = np.array([rrear_x - com_x, rrear_y - com_y])

        eps = 1 / self.I * (F1_x * r_front_diff[1] - F1_y * r_front_diff[0]) + 1 / self.I * (
            F2_x * r_rear_diff[1] - F2_y * r_rear_diff[0]
        )

        out = casadi.vertcat(ax, ay, eps)

        self.aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [out])

    def q_int(self, q, v, v_dot, u, dt):
        q_out = q
        q_out[:3] = q[:3] + v * dt
        q_out[3:] = q[3:] + u[4:] * dt
        return q_out

    def v_int(self, q, v, v_dot, u, dt):
        v_out = v
        v_out += v_dot * dt
        return v_out

    def get_value(self, q, v, u):
        return self.aba_fn(q, v, u)

    def com_position(self, q):
        x, y = q[0], q[1]
        return x, y

    def front_leg_position(self, q):
        lib = np if isinstance(q, np.ndarray) else casadi
        com_x, com_y = self.com_position(q)
        phi = q[2]
        theta1 = q[3]
        theta2 = q[4]

        shoulder_front_x = com_x + self.L / 2 * lib.cos(phi)
        shoulder_front_y = com_y + self.L / 2 * lib.sin(phi)

        knee_front_x = shoulder_front_x + self.leg_length * lib.cos(phi - np.pi / 2 * 1 + theta1)
        knee_front_y = shoulder_front_y + self.leg_length * lib.sin(phi - np.pi / 2 * 1 + theta1)

        ankle_front_x = knee_front_x + self.leg_length * lib.cos(phi - np.pi / 2 * 1 + theta1 + theta2)
        ankle_front_y = knee_front_y + self.leg_length * lib.sin(phi - np.pi / 2 * 1 + theta1 + theta2)

        return ankle_front_x, ankle_front_y

    def rear_leg_position(self, q):
        lib = np if isinstance(q, np.ndarray) else casadi
        com_x, com_y = self.com_position(q)
        phi = q[2]
        theta3 = q[5]
        theta4 = q[6]

        shoulder_rear_x = com_x - self.L / 2 * lib.cos(phi)
        shoulder_rear_y = com_y - self.L / 2 * lib.sin(phi)

        knee_rear_x = shoulder_rear_x + self.leg_length * lib.cos(phi - np.pi / 2 * 1 + theta3)
        knee_rear_y = shoulder_rear_y + self.leg_length * lib.sin(phi - np.pi / 2 * 1 + theta3)

        ankle_rear_x = knee_rear_x + self.leg_length * lib.cos(phi - np.pi / 2 * 1 + theta3 + theta4)
        ankle_rear_y = knee_rear_y + self.leg_length * lib.sin(phi - np.pi / 2 * 1 + theta3 + theta4)

        return ankle_rear_x, ankle_rear_y

In [4]:
m, L, leg_length = 1, 1, 0.3
I = m * L**2 / 12
dynamics = QuadrupedDynamics(m, I, L, leg_length)
options = ncsudyn.optimize.TrajectoryOptimizerOptions(T=1, dt=0.02, nq=dynamics.nq, nv=dynamics.nv, nu=dynamics.nu)
optimizer = ncsudyn.optimize.TrajectoryOptimizer(options)

optimizer.add_dynamics(dynamics)
optimizer.optimize()

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2100
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1350

Total number of variables............................:      910
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      500
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  